In [11]:
# import dependencies
from config import (gkey)
import gmaps
import gmaps.datasets
from ipywidgets.embed import embed_minimal_html
import numpy as np
import pandas as pd
import requests
import time
import random
from random import sample

# Google API Key
from config import gkey

In [12]:
# import Crash DC csv
crashes_df = pd.read_csv("Resources/Crashes_in_DC.csv", dtype="object", encoding="utf-8")
reduced_df = crashes_df[['LATITUDE', 'LONGITUDE', 'WARD', 'MAJORINJURIES_BICYCLIST', 'FATAL_BICYCLIST', 'MAJORINJURIES_DRIVER',
       'MINORINJURIES_DRIVER', 'UNKNOWNINJURIES_DRIVER', 'FATAL_DRIVER',
       'MAJORINJURIES_PEDESTRIAN', 'MINORINJURIES_PEDESTRIAN',
       'UNKNOWNINJURIES_PEDESTRIAN', 'FATAL_PEDESTRIAN', 'TOTAL_VEHICLES',
       'TOTAL_BICYCLES', 'TOTAL_PEDESTRIANS']]
# print(len(reduced_df))
# reduced_df.head()

In [14]:
crashes_df.head()

,X,Y,OBJECTID,CRIMEID,CCN,REPORTDATE,ROUTEID,MEASURE,OFFSET,STREETSEGID,...,NEARESTINTROUTEID,NEARESTINTSTREETNAME,OFFINTERSECTION,INTAPPROACHDIRECTION,LOCATIONERROR,LASTUPDATEDATE,MPDLATITUDE,MPDLONGITUDE,MPDGEOX,MPDGEOY
0,-77.027055800100001,38.894744341799999,84817311,25166250,14159833,2014-10-17T05:00:00.000Z,11001102,0,1.1e-05,12987,...,11069812,PENNSYLVANIA AVE NW,0.8508568,North,NaN,NaN,NaN,NaN,NaN,NaN
1,-77.031465893900005,38.945669158299999,84817312,25078515,14019182,2014-02-09T05:00:00.000Z,11011052,854.964,0,6975,...,11018782,BUCHANAN ST NW,53.8418396,South,NaN,NaN,NaN,NaN,NaN,NaN
2,-76.9394791032,38.895598944600003,84817313,27671073,18074797,2018-05-10T04:46:33.000Z,12031522,18.55,20.778688,1575,...,12004402,44TH ST NE,7.6525486,West,NaN,2018-05-30T14:31:55.000Z,38.895597,-76.939124,405253.700000,136438.130000
3,-77.041127710699996,38.905671268100001,84817314,27653615,18061995,2018-04-22T16:22:24.000Z,11057852,0,0.004827,10134,...,11025152,CONNECTICUT AVE NW,0.8401932,North,NaN,2018-05-25T17:37:32.000Z,38.905664,-77.041125,396432.910000,137534.800000
4,-77.083535021200007,38.956476342400002,84817315,25078516,14018944,2014-02-08T05:00:00.000Z,11094052,6439.51,0,422,...,11039712,GARRISON ST NW,68.7466755,North,NaN,NaN,38.956366,-77.083862,NaN,NaN


In [13]:
clean_df = reduced_df.dropna()
print(len(clean_df))
clean_df.head()

204265


,LATITUDE,LONGITUDE,WARD,MAJORINJURIES_BICYCLIST,FATAL_BICYCLIST,MAJORINJURIES_DRIVER,MINORINJURIES_DRIVER,UNKNOWNINJURIES_DRIVER,FATAL_DRIVER,MAJORINJURIES_PEDESTRIAN,MINORINJURIES_PEDESTRIAN,UNKNOWNINJURIES_PEDESTRIAN,FATAL_PEDESTRIAN,TOTAL_VEHICLES,TOTAL_BICYCLES,TOTAL_PEDESTRIANS
0,38.894737,-77.027054,Ward 2,0,0,0,0,0,0,0,1,0,0,1,0,1
1,38.945662,-77.031464,Ward 4,0,0,1,0,0,0,0,0,0,0,2,0,0
2,38.895591,-76.939477,Ward 7,0,0,0,2,0,0,0,0,0,0,2,0,0
3,38.905664,-77.041125,Ward 2,0,0,0,1,0,0,0,0,0,0,2,0,0
4,38.956366,-77.083862,Ward 3,0,0,0,0,0,0,0,0,0,0,2,0,0


In [4]:
major_drivers = clean_df[(clean_df["MAJORINJURIES_DRIVER"] > "0")]
major_driver_loc = major_drivers[["LATITUDE", "LONGITUDE"]].astype(float)
print(len(major_drivers))
n = int(len(major_drivers)*.03)
print(n)

sample_major_drivers = major_driver_loc.sample(n=n, random_state=1)
# sample_major_drivers.head()

14803
444


In [5]:
minor_drivers = clean_df[(clean_df["MINORINJURIES_DRIVER"] > "0")]
minor_driver_loc = minor_drivers[["LATITUDE", "LONGITUDE"]].astype(float)
print(len(minor_drivers))
n = int(len(minor_drivers)*.03)
print(n)

sample_minor_drivers = minor_driver_loc.sample(n=n, random_state=1)

type(sample_minor_drivers)
# sample_minor_drivers.head()

30916
927


pandas.core.frame.DataFrame

In [6]:
# DON'T SAMPLE fatal drivers. Count is already low.
fatal_drivers = clean_df[(clean_df["FATAL_DRIVER"] > "0")]
fatal_driver_loc = fatal_drivers[["LATITUDE", "LONGITUDE"]].astype(float)
print(len(fatal_drivers))
n = int(len(fatal_drivers)*.05)
print(n)

# sample_fatal_drivers = fatal_driver_loc.sample(n=n, random_state=1)
# sample_minor_drivers.head()
len(fatal_driver_loc)

161
8


161

## Map results

In [7]:
gmaps.configure(api_key=gkey)

In [8]:
# color codes
major_color = '#ab47bc' # purple
minor_color = '#26a69a' # teal
fatal_color = '#dd2c00' # red

In [9]:
# Target city
target_city = "Washington, DC"

# Build the endpoint URL
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city, gkey)

#### 3% sample of Major & Minor Injuries
#### Full count of fatal injuries

In [10]:
major_marker = gmaps.symbol_layer(
    sample_major_drivers, fill_color= major_color, 
    stroke_color=None, stroke_opacity=0, scale=4)
minor_marker = gmaps.symbol_layer(
    sample_minor_drivers, fill_color= minor_color, 
    stroke_color=None, stroke_opacity=0, scale=4)
fatal_marker = gmaps.symbol_layer(
    fatal_driver_loc, fill_color= fatal_color, 
    stroke_color=None, stroke_opacity=0, scale=4)

fig = gmaps.figure()
fig.add_layer(major_marker)
fig.add_layer(minor_marker)
fig.add_layer(fatal_marker)

embed_minimal_html('driver_injuries.html', views=[fig])
fig

Figure(layout=FigureLayout(height='420px'))